In [1]:
from __future__ import annotations
import re
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torchvision.transforms import InterpolationMode
from torchmetrics.functional import accuracy as tm_accuracy

/home/skitarii/PycharmProjects/mipt-masters/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [2]:
# 1. Автоматическое определение колонок
_PIXEL_RE = re.compile(r"pixel(\d+)", flags=re.IGNORECASE)

def _detect_columns(df: pd.DataFrame) -> Tuple[str, str, Sequence[str]]:
    """
    Находит названия ключевых столбцов в CSV-фиде.

    Parameters
    ----------
    df : pd.DataFrame
        Загруженный датафрейм (train или test).

    Returns
    -------
    label_col : str
        Имя столбца с метками классов (0–9). Может быть None в тесте.
    id_col : str
        Имя столбца с уникальным идентификатором строки. Может быть None.
    pixel_cols : list[str]
        Список из 784 имён столбцов вида pixel0 … pixel783 в правильном порядке.

    Raises
    ------
    ValueError
        Если найдено не ровно 784 столбца-пикселя.
    """
    # Приводим имена к lower-case, чтобы быть нечувствительными к регистру
    lower = {c.lower(): c for c in df.columns}
    label_col = lower.get("label")
    id_col = lower.get("id")
    # Все столбцы, которые подходят под шаблон pixel\d+
    pixel_cols = [c for c in df.columns if _PIXEL_RE.match(c)]
    # Сортируем по номеру пикселя, чтобы получить правильный порядок 0…783
    pixel_cols.sort(key=lambda x: int(_PIXEL_RE.match(x).group(1)))
    # Обязательно должны быть все 784 пикселя (28 × 28)
    if len(pixel_cols) != 784:
        raise ValueError(f"Expected 784 pixel cols, got {len(pixel_cols)}")
    return label_col, id_col, pixel_cols


# 2. Очистка и приведение данных
def _clean(df: pd.DataFrame, is_train: bool = True) -> pd.DataFrame:
    """
    Преобразует «сырае» CSV-данные к числовому виду без пропусков.

    1. Преобразует все pixel-значения к float32, заменяя нечисловые на 0.
    2. Ограничивает допустимый диапазон [0, 255].
    3. Для тренировочных данных:
       * удаляет строки без label,
       * обрезает метки вне диапазона 0–9.
    4. Сбрасывает индекс.

    Parameters
    ----------
    df : pd.DataFrame
        Исходные данные.
    is_train : bool
        True — это обучающая выборка с колонкой label.

    Returns
    -------
    pd.DataFrame
        Очищенный датафрейм, готовый к использованию в Dataset.
    """
    label_col, _, pixel_cols = _detect_columns(df)
    df[pixel_cols] = (
        df[pixel_cols]
        .apply(pd.to_numeric, errors="coerce")
        .fillna(0)
        .clip(0, 255)
        .astype(np.float32)
    )
    if is_train:
        df = df.dropna(subset=[label_col])
        df[label_col] = df[label_col].astype(int)
        df = df[df[label_col].between(0, 9)]
    return df.reset_index(drop=True)


# 3. PyTorch-совместимый Dataset
class FashionCSVDataset(Dataset):
    """
    Dataset для Fashion-MNIST, который читает изображения прямо из CSV.

    Параметры
    ---------
    df : pd.DataFrame
        Предварительно очищенный датафрейм (см. `_clean`).
    training : bool
        True, если у строк есть метки (колонка label).
    transform : Callable | None
        Аугментации/препроцесс, совместимые с torchvison.transforms.
        Ожидают tensor формата (C, H, W).
    """
    def __init__(self, df: pd.DataFrame, *, training: bool, transform=None):
        label_col, id_col, pixel_cols = _detect_columns(df)
        self.transform = transform
        # Ids: если столбца нет, генерируем индексы 0..N-1
        self.ids = df[id_col].astype(int).to_numpy() if id_col else np.arange(len(df))
        # Целевые метки (None в тесте)
        self.targets = df[label_col].astype(int).to_numpy() if training else None
        # Массив изображений в формате (N, 28, 28) и типе float32
        self.images = (df[pixel_cols].to_numpy().reshape(-1, 28, 28) / 255.0).astype(np.float32)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx: int):
        """
        Возвращает тензор изображения и либо:
        * метку (train)   — (x, y)
        * id (inference)  — (x, id)
        """
        x = torch.from_numpy(self.images[idx]).unsqueeze(0)  # (1,28,28)
        # Аугментации
        if self.transform:
            x = self.transform(x)
        if self.targets is None:
            return x, int(self.ids[idx])
        return x, int(self.targets[idx])

# Model

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            # Block 1 — 32 ch
            nn.Conv2d(1, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(inplace=True),
            nn.MaxPool2d(2), nn.Dropout2d(0.25),
            # Block 2 — 64 ch
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.MaxPool2d(2), nn.Dropout2d(0.25),
            # Block 3 — 128 ch
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(inplace=True),
            nn.MaxPool2d(2), nn.Dropout2d(0.25),
            # Global pooling
            nn.AdaptiveAvgPool2d(1),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 256), nn.ReLU(inplace=True), nn.Dropout(0.5),
            nn.Linear(256, 10),
        )

    def forward(self, x):
        """
        Parameters
        ----------
        x : torch.Tensor
            Входной батч формы (B, 1, 28, 28)

        Returns
        -------
        torch.Tensor
            Логиты размера (B, 10)
        """
        x = self.features(x)     # извлекаем признаковое описание
        x = self.classifier(x)   # предсказываем класс
        return x

# Trainer

In [4]:
def train_one_epoch(model, loader, criterion, optimizer, scheduler, device):
    """
    Один проход по всему обучающему набору.

    * Переключаем сеть в train-режим (`model.train()`) — активируются Dropout и
      BatchNorm собирает статистику.
    * Для каждого mini-batch:
        1. Переносим данные на GPU/CPU.
        2. Обнуляем градиенты (`optimizer.zero_grad()`).
        3. Считаем loss и back-prop (`loss.backward()`).
        4. Обновляем веса (`optimizer.step()`).
        5. Делаем шаг LR-плана (`scheduler.step()`);  
    * Возвращаем средний loss за эпоху, чтобы мониторить кривую обучения.
    """
    model.train(); epoch_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        loss = criterion(model(x), y)
        loss.backward()
        optimizer.step()
        scheduler.step()
        epoch_loss += loss.item() * y.size(0)
    return epoch_loss / len(loader.dataset)


@torch.no_grad()
def evaluate(model, loader, device):
    """
    Оценка точности (Accuracy) на валидационном лоадере.
    """
    model.eval(); preds, targets = [], []
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        preds.append(model(x)); targets.append(y)
    preds = torch.cat(preds); targets = torch.cat(targets)
    return tm_accuracy(preds, targets, task='multiclass', num_classes=10).item()


def fit(model, tr_loader, val_loader, device, *, epochs=30, patience=8, lr=3e-3):
    """
    Полный цикл обучения + ранняя остановка (Early Stopping).

    Параметры
    ---------
    model : nn.Module
        Наша CNN / DenseNet.
    tr_loader, val_loader : DataLoader
        Лоадеры для train и validation.
    device : torch.device
        'cuda' или 'cpu'.
    epochs : int
        Максимальное число эпох.
    patience : int
        Сколько эпох подряд можно не улучшаться прежде чем остановиться.
    lr : float
        Пиковый learning rate для OneCycle.

    Возвращает
    ----------
    nn.Module
        Лучшая (по val-accuracy) версия модели, загруженная из чекпойнта.
    """
    criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr, epochs=epochs, steps_per_epoch=len(tr_loader), pct_start=0.3)

    best, n_bad, ckpt = 0.0, 0, Path('best_model.pt')
    for ep in range(1, epochs + 1):
        tr_loss = train_one_epoch(model, tr_loader, criterion, optimizer, scheduler, device)
        val_acc = evaluate(model, val_loader, device)
        if val_acc > best + 1e-4:
            best, n_bad = val_acc, 0; torch.save(model.state_dict(), ckpt)
        else:
            n_bad += 1
        print(f'Epoch {ep:02d}: loss={tr_loss:.4f}  val_acc={val_acc:.4f}  best={best:.4f}')
        if n_bad >= patience:
            print(f'Early stopping (patience={patience}) at epoch {ep}'); break

    model.load_state_dict(torch.load(ckpt)); return model

# Load data

In [5]:
train_df = _clean(pd.read_csv("fmnist_train.csv"), is_train=True)
test_df  = _clean(pd.read_csv("fmnist_test.csv"),  is_train=False)

In [6]:
assert train_df.isna().sum().sum() == 0, "NaN present in train"  # quick sanity
assert test_df.isna().sum().sum() == 0, "NaN present in test"

In [7]:
train_df.shape

(17040, 786)

In [8]:
test_df.shape

(10000, 785)

# Make dataloaders

In [9]:
batch_size = 256
epochs = 150
patience = 10

In [10]:
label_col = [c for c in train_df.columns if c.lower() == 'label'][0]
tr_df, val_df = train_test_split(train_df, test_size=0.1, stratify=train_df[label_col], random_state=42)

aug = transforms.Compose([
    transforms.RandomCrop(28, padding=4),
    transforms.RandomRotation(10, interpolation=InterpolationMode.BILINEAR),
    transforms.RandomAffine(0, translate=(0.08, 0.08), scale=(0.95, 1.05)),
    transforms.RandomHorizontalFlip(0.4),
    transforms.RandomErasing(p=0.10, scale=(0.02, 0.12)),
])


train_ds = FashionCSVDataset(tr_df, training=True, transform=aug)
val_ds   = FashionCSVDataset(val_df, training=True, transform=None)

dl_kwargs = dict(batch_size=batch_size, num_workers=2, pin_memory=True)
train_loader = DataLoader(train_ds, shuffle=True, **dl_kwargs)
val_loader   = DataLoader(val_ds, shuffle=False, **dl_kwargs)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
model = fit(model, train_loader, val_loader, device, epochs=epochs, patience=patience)

Epoch 01: loss=2.1946  val_acc=0.3944  best=0.3944
Epoch 02: loss=1.8224  val_acc=0.6056  best=0.6056
Epoch 03: loss=1.5348  val_acc=0.6696  best=0.6696
Epoch 04: loss=1.3436  val_acc=0.7101  best=0.7101
Epoch 05: loss=1.1929  val_acc=0.7383  best=0.7383
Epoch 06: loss=1.1029  val_acc=0.7653  best=0.7653
Epoch 07: loss=1.0193  val_acc=0.7612  best=0.7653
Epoch 08: loss=0.9827  val_acc=0.7817  best=0.7817
Epoch 09: loss=0.9426  val_acc=0.8052  best=0.8052
Epoch 10: loss=0.9174  val_acc=0.7876  best=0.8052
Epoch 11: loss=0.8812  val_acc=0.7934  best=0.8052
Epoch 12: loss=0.8697  val_acc=0.8116  best=0.8116
Epoch 13: loss=0.8534  val_acc=0.8263  best=0.8263
Epoch 14: loss=0.8302  val_acc=0.8128  best=0.8263
Epoch 15: loss=0.8196  val_acc=0.8322  best=0.8322
Epoch 16: loss=0.8034  val_acc=0.8228  best=0.8322
Epoch 17: loss=0.7990  val_acc=0.8410  best=0.8410
Epoch 18: loss=0.7865  val_acc=0.8545  best=0.8545
Epoch 19: loss=0.7679  val_acc=0.8369  best=0.8545
Epoch 20: loss=0.7589  val_acc=

# Inference

In [12]:
test_ds = FashionCSVDataset(test_df, training=False, transform=None)
test_loader = DataLoader(
    test_ds,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True,
)

model.eval()
all_ids: list[int] = []
all_preds: list[int] = []
with torch.no_grad():
    for x, ids in test_loader:
        logits = model(x.to(device))
        all_preds.extend(logits.argmax(1).cpu().tolist())
        all_ids.extend(ids.tolist())

submission = pd.DataFrame({"id": all_ids, "label": all_preds})
submission.to_csv("sample_submission.csv", index=False)
print(f"Saved sample_submission.csv (rows={len(submission)})")

Saved sample_submission.csv (rows=10000)
